In [747]:
""" Prepare a datasetof 5 objects with all kind of attributes seen
    Nominal
    Symmetric Binary
    Asymmetric Binary
    Ordinal
    Numeric
    
Compute the dissimilarity matrix for each of the attribute separately
Compute the dissimilarity for the aggregate case
    """
    


import numpy as np
from scipy import sparse,spatial
from random import Random, randint, random, shuffle
from secrets import choice
import pandas as pd




In [748]:



class Person():
    """ A class for a person with a name, gender and and age. \n
        Males are represented with 1/True and females with 0/False \n
        If no age is specified a random age between 20 and 90 is assigned. \n
        You can add a variety of attributes with the different add methods.
    """
    def __init__(self, name, sex, age = None):
        self.all_attributes = {}
        self.name = name # Nominal
        self.add_nominal('Name',name)
        self.sex = sex
        self.add_Symmetric_binary('Sex',sex)
        
        if age:self.age = age # Ordinal
        else: self.age = randint(20,90)
        self.add_ordinal('Age',self.age)
        
       
        
        
        
    def add_nominal(self,name:str,value):
        if 'nominal' not in (self.__dir__()):
            self.nominal = {name:value}
            self.all_attributes.update(self.nominal)
        else: 
            self.nominal[name]=value
            self.all_attributes.update(self.nominal)
            
    
    def add_Asymmetric_binary(self,name:str,value:bool):
        if 'asymmetric_binary' not in (self.__dir__()):
            self.asymmetric_binary = {name:value}
            self.all_attributes.update(self.asymmetric_binary)
        else: 
            self.asymmetric_binary[name]=value
            self.all_attributes.update(self.asymmetric_binary)
        
    
    def add_Symmetric_binary(self,name:str,value:bool):
        if 'symetric_binary' not in (self.__dir__()):
            self.symmetric_binary = {name:value}
            self.all_attributes.update(self.symmetric_binary)
        else: 
            self.symmetric_binary[name]=value
            self.all_attributes.update(self.symmetric_binary)
    
    def add_ordinal(self,name:str,value):
        if 'ordinal' not in (self.__dir__()):
            self.ordinal = {name:value}
            self.all_attributes.update(self.ordinal)
        else: 
            self.ordinal[name]=value
            self.all_attributes.update(self.ordinal)
            
    def add_numeric(self,name:str,value:float):
        if 'numeric' not in (self.__dir__()):
            self.numeric = {name:value}
            self.all_attributes.update(self.numeric)
        else: 
            self.numeric[name]=value
            self.all_attributes.update(self.numeric)
    
    def get_attributes(self):
        return(self.all_attributes)
    
    ''' def __str__(self):
        sex = 'male' if self.sex == 1 else 'female'
        return f'{self.name} is a {sex} at the age of {self.age}.'
         '''

In [749]:
# Load some names and the sex of the names and then som cities in the US
import random
persons = pd.read_csv('name_gender.csv')
jobs = pd.read_table('jobs.txt')
cities_loaded = pd.read_csv('cities.csv')
cities = list(cities_loaded['City'].dropna())

n = 10 # Number of persons you want to create
people = []
# in this loop the persons are created and assign a name and gender, which is a nominal and symmetric attribute
for i in range(n):
    r = randint(1,len(persons)-2)
    people.append(Person(persons['name'][r],1 if persons['gender'][r]== 'M' else 0))


# Add some attributes 
for p in people:
    # Add employment as an asymmetric boolean attribute with True as valued higher
    p.add_Asymmetric_binary('Employed',random.choice([1, 0]))
    # Add city of residence as nominal attribute
    p.add_nominal('City',cities[randint(1,len(cities)-1)])
    # Add a job as nominal
    p.add_nominal('Job', jobs['jobs'][random.randint(0,len(jobs)-1)] if p.asymmetric_binary['Employed']==True else None)
    # Add an ordinal salary depending on employment, randomized
    p.add_ordinal('Yearly Salary in USD',round(random.uniform(10000,100000)) if p.asymmetric_binary['Employed']==True else 0)
    # Add 2 numeric attribute, height and weight
    p.add_numeric('Height in cm',np.random.normal(171) if p.symmetric_binary['Sex']==1 else np.random.normal(159.5))
    # Assign a weight around the height using the devine formula
    p.add_numeric('Weight in kg',np.random.normal(50+0.9*(p.numeric['Height in cm']-152)) if p.symmetric_binary['Sex']==1 else np.random.normal(45.5+0.9*(p.numeric['Height in cm']-152)))
    # Add symmetric attribute 'Parent', true or false depending on if they have kids, randomized
    p.add_Symmetric_binary('Parent',random.choice([1, 0]))
    # Add number of kids
    p.add_numeric('N of kids', random.randint(1,7) if p.symmetric_binary['Parent'] == True else 0)
    # Add symmetric attribute 'Married', true or false depending on if they have kids, randomized
    p.add_Symmetric_binary('Married', random.choice([1, 0]))
    # Add asymmetric attribute for drivers license, with True being valued higher
    p.add_Asymmetric_binary('Drivers license', random.choice([1, 0]))
     # Add asymmetric attribute if the person has been convicted of a crime, 1 being never, thus valued higher
    p.add_Asymmetric_binary('Convicted fellon', random.choice([1, 0]))
    
    
    
    
   

In [750]:



from scipy.spatial import distance


def dissimilartiy(p:list[Person],kind:str):
    """ Function for calculating dissimilarity matrix

    Args:
        people (list of Person objects): the persons to be compared \n
        kind (string): Can be one of the following: \n
        ['ordinal', 'asymmetric', 'symmetric', 'binary', 'nominal', 'numeric', 'congruent'] \n
        return: prints a dissimilarity matrix
    """
    def nominal(p):
        d = np.zeros((len(p[0].nominal),len(p),len(p)))
        for k,n in enumerate(p[0].nominal): # loop over the different nominal values eg names or cities k is index, n is nalue
            label = []
            for i in range(0,len(p)):
                for j in range(0,len(p)):
                    d[k,i,j] = 0 if p[i].nominal[n] == p[j].nominal[n] else 1
                
       
        # Create the summarized matrix
        d_nominal = np.zeros((len(d[0]),len(d[0])))
        
        if len(d) == 1: df = pd.DataFrame(d[0,:,:])
        else:
            for n in range(len(d)):
                for i in range(len(d[n])):
                    for j in range(len(d[n])):
                        d_nominal[i,j] += d[n,i,j]
            
            
            df = pd.DataFrame(d_nominal / len(d))
        return df
        
    if kind.lower() == 'nominal':
        print(nominal(p))
        
    def asymmetric(p):
        # Works the same way as the nominal
        d = np.zeros((len(p[0].asymmetric_binary),len(p),len(p)))
        for k,n in enumerate(p[0].asymmetric_binary): # loop over the different asymetrical values eg employed or convicted k is index, n is nalue
            label = []
            for i in range(0,len(p)):
                for j in range(0,len(p)):
                    d[k,i,j] = 0 if p[i].asymmetric_binary[n] == p[j].asymmetric_binary[n] else 1


        # Create the summarized matrix
        d_asymmetric = np.zeros((len(d[0]),len(d[0])))
        if len(d) == 1: df = pd.DataFrame(d[0,:,:])
        
        else:
            
            for n in range(len(d)):
                for i in range(len(d[n])):
                    for j in range(len(d[n])):
                        d_asymmetric[i,j] += d[n,i,j]
            df = pd.DataFrame(d_asymmetric / len(d))
        return df
            
    if kind.lower() == 'asymmetric':
       print(asymmetric(p))
    
    def symmetric(p):
    # Works the same way as the nominal
        d = np.zeros((len(p[0].symmetric_binary),len(p),len(p)))
        for k,n in enumerate(p[0].symmetric_binary): # loop over the different asymetrical values eg employed or convicted k is index, n is nalue
            
            for i in range(0,len(p)):
                for j in range(0,len(p)):
                    d[k,i,j] = 0 if p[i].symmetric_binary[n] == p[j].symmetric_binary[n] else 1


        # Create the summarized matrix
        d_symmetric = np.zeros((len(d[0]),len(d[0])))
        
        if len(d) == 1: df = pd.DataFrame(d[0,:,:])
        
        else:
            for n in range(len(d)):
                for i in range(len(d[n])):
                    for j in range(len(d[n])):
                        d_symmetric[i,j] += d[n,i,j]
            df = pd.DataFrame(d_symmetric / len(d))
        return df
    
    if kind.lower() == 'symmetric':
        print(symmetric(p))
    
    def binary(p):
        df_asymmetric = asymmetric(p)
        df_symmetric = symmetric(p)
       
        df_binary = df_asymmetric.add(df_symmetric)/2
        
        return df_binary
    
    if kind.lower() == 'binary':
        keys = p[0].asymmetric_binary.keys()
        print(f'for the values {keys} the dissimilarity matrix looks like the following:')
        print(binary(p))
        
                
    def ordinal(p):
        # Works the same way as the nominal
        d = np.zeros((len(p[0].ordinal),len(p),len(p)))
        df = pd.DataFrame.from_dict(p[x].ordinal for x in range(len(p)))
        df_list = []
        
        # seperate the different ordinal values into their own dataframe
        for x in range(len(df.columns)):
            df_list.append(df.iloc[:, x])
        
        # Sort each datafram
        sorted_df = []
        for x in range(len(df_list)):
            sorted_df.append(df_list[x].sort_values())
        
        for x in range(len(p[0].ordinal)):
            z = []
            for r in range(len(p)):
                z.append(r/(len(p)-1))
            sorted_df[x]['Rank'] = z
        

        # Compute distance in a nmatrix (Dissimilarity)
        def distance(a,b):
            return np.abs(b-a)
        
        for k,n in enumerate(sorted_df):
            for i in n.index[:-1]:
                for j in n.index[:-1]:
                    d[k,i,j] = distance(n['Rank'][i],n['Rank'][j])
        
        
        # Return summed matrix
        d_ordinal = np.zeros((len(d[0]),len(d[0])))
        if len(d) == 1: df = pd.DataFrame(d[0,:,:])
        
        else:
            for n in range(len(d)):
                for i in range(len(d[n])):
                    for j in range(len(d[n])):
                        d_ordinal[i,j] += d[n,i,j]
            df = pd.DataFrame(d_ordinal / len(d))
        return df

    
    if kind.lower() == 'ordinal':
        keys = p[0].ordinal.keys()
        print(f'for the values {keys} the dissimilarity matrix looks like the following:')
        print(ordinal(p))
    
    def numeric(p):
        # Uses the normalised distance
        d = np.zeros((len(p[0].numeric),len(p),len(p)))
        # Take out h_max/min and their indexes

        
        for k,n in enumerate(p[0].numeric): # loop over the different numerical values, k is index, n is nalue
            # vectors for simplicity
            u = [x.numeric[n] for x in p]
            var = np.var(u)
            for i in range(0,len(p)):
                for j in range(0,len(p)):
                    d[k,i,j] = (np.abs(u[i]-u[j]))/var

        # Create the summarized matrix
        d_numerical = np.zeros((len(d[0]),len(d[0])))
        
        if len(d) == 1: df = pd.DataFrame(d[0,:,:])
        
        else:
            for n in range(len(d)):
                for i in range(len(d[n])):
                    for j in range(len(d[n])):
                        d_numerical[i,j] += d[n,i,j]
            df = pd.DataFrame(d_numerical/len(d))
        return df
    
    if kind.lower() == 'numeric':
        print(numeric(p))    
    
    if kind.lower() == 'congruent':
        ordinal_df = ordinal(p)
        binary_df = binary(p)
        nominal_df = nominal(p)
        numeric_df = numeric(p)
        congruent_df = ordinal_df.add(binary_df).add(nominal_df).add(numeric_df)/4
        print(congruent_df)


dissimilartiy(people,kind = 'congruent')

          0         1         2         3         4         5         6  \
0  0.000000  0.465509  0.410755  0.605981  0.461811  0.490686  0.680147   
1  0.465509  0.000000  0.364948  0.557740  0.526074  0.366664  0.586907   
2  0.410755  0.364948  0.000000  0.533197  0.502022  0.420069  0.638626   
3  0.605981  0.557740  0.533197  0.000000  0.472702  0.562692  0.532934   
4  0.461811  0.526074  0.502022  0.472702  0.000000  0.475961  0.564376   
5  0.490686  0.366664  0.420069  0.562692  0.475961  0.000000  0.470242   
6  0.680147  0.586907  0.638626  0.532934  0.564376  0.470242  0.000000   
7  0.660791  0.556686  0.611075  0.502714  0.647150  0.357673  0.335776   
8  0.662711  0.691687  0.665257  0.471048  0.558988  0.583640  0.506331   
9  0.564890  0.611849  0.496902  0.641210  0.554479  0.411851  0.656769   

          7         8         9  
0  0.660791  0.662711  0.564890  
1  0.556686  0.691687  0.611849  
2  0.611075  0.665257  0.496902  
3  0.502714  0.471048  0.641210  
4  0